In [1]:
# import, 데이터 불러오기
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as fm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, classification_report, make_scorer,confusion_matrix
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings(action = 'ignore')

In [2]:
test = pd.read_csv('aug_test.csv')
train = pd.read_csv('aug_train.csv')
submission = pd.read_csv('sample_submission.csv')

In [3]:
# 한글 폰트 처리# 한글 폰트 처리
font_name = fm.FontProperties(fname='C:\\Windows\\Fonts\\malgun.ttf').get_name()
mpl.rcParams["font.family"] = font_name
mpl.rcParams["font.size"] = 15
mpl.rcParams['xtick.labelsize'] = 12
mpl.rcParams['ytick.labelsize'] = 12
mpl.rcParams['axes.labelsize'] = 15
mpl.rcParams['axes.unicode_minus'] = False

In [4]:
encoding_columns = ['city', 'gender','relevent_experience', 'enrolled_university', 'education_level', 'major_discipline', 'experience', 'company_size', 'company_type', 'last_new_job']
not_encoding_columns = ['city_development_index','training_hours']

# 각 컬럼(Feature)마다 변환전의 고유값을 저장할 딕셔너리
enc_dict = {}

def encoding_label(x):
    le = LabelEncoder()
    # x.astype(str)로 error 해결
    r = le.fit_transform(x.astype(str))
    enc_dict[x.name] = le.classes_
    return r

for en in encoding_columns:
    encoding_label(train[en])
# print(enc_dict)

- https://rfriend.tistory.com/267
- gender                    4508(3)
- enrolled_university        386(3)
- education_level            460(5)
- major_discipline          2813(6)
- experience                  65(22)
- company_size              5938(8)
- company_type              6140(6)
- last_new_job               423(6)

In [5]:
len(train['gender'].unique())

4

In [6]:
new_train = train.drop(['enrollee_id'],axis=1)
label_encoding_data = new_train[encoding_columns].apply(encoding_label)

In [7]:
label_encoding_data['gender'].unique()

array([1, 3, 0, 2])

In [8]:
# print(encoding_columns)

# 마지막 라벨인 결측값을 np.nan으로 변경
for ec in encoding_columns:
    nonan = len(train[ec].unique()) - 1
    label_encoding_data[ec] = label_encoding_data[ec].replace(nonan, np.nan)

In [9]:
label_encoding_data['gender'].unique()

array([ 1., nan,  0.,  2.])

In [10]:
from sklearn.impute import KNNImputer # 수치형 데이터만 가능
# 범주형 데이터는 레이블 인코딩 후 사용 가능

imputer = KNNImputer(n_neighbors = 1)
data_filled = imputer.fit_transform(label_encoding_data)

In [11]:
new_train = pd.DataFrame(data_filled, columns=encoding_columns)
new_train.head()

city  gender  relevent_experience  enrolled_university  education_level  \
0   5.0     1.0                  0.0                  0.0              0.0   
1  77.0     1.0                  0.0                  0.0              0.0   
2  64.0     1.0                  0.0                  0.0              0.0   
3  14.0     1.0                  0.0                  2.0              0.0   
4  50.0     1.0                  0.0                  0.0              2.0   

   major_discipline  experience  company_size  company_type  last_new_job  
0               5.0        21.0           1.0           1.0           0.0  
1               5.0         6.0           4.0           5.0           4.0  
2               5.0        15.0           2.0           5.0           0.0  
3               1.0        20.0           3.0           5.0           1.0  
4               5.0        21.0           4.0           1.0           3.0

In [12]:
train.columns

Index(['enrollee_id', 'city', 'city_development_index', 'gender',
       'relevent_experience', 'enrolled_university', 'education_level',
       'major_discipline', 'experience', 'company_size', 'company_type',
       'last_new_job', 'training_hours', 'target'],
      dtype='object')

In [13]:
new_train = pd.concat([new_train, train['city_development_index'],train['training_hours']],axis=1)
new_train

city  gender  relevent_experience  enrolled_university  \
0       5.0     1.0                  0.0                  0.0   
1      77.0     1.0                  0.0                  0.0   
2      64.0     1.0                  0.0                  0.0   
3      14.0     1.0                  0.0                  2.0   
4      50.0     1.0                  0.0                  0.0   
...     ...     ...                  ...                  ...   
19153  55.0     1.0                  0.0                  0.0   
19154   5.0     1.0                  0.0                  0.0   
19155   5.0     1.0                  0.0                  1.0   
19156  94.0     1.0                  0.0                  0.0   
19157  95.0     1.0                  0.0                  0.0   

       education_level  major_discipline  experience  company_size  \
0                  0.0               5.0        21.0           1.0   
1                  0.0               5.0         6.0           4.0   
2                  0.0               5.0        15.0           2.0   
3                  0.0               1.0        20.0           3.0   
4                  2.0               5.0        21.0           4.0   
...                ...               ...         ...           ...   
19153              0.0               2.0         5.0           2.0   
19154              0.0               5.0         5.0           4.0   
19155              0.0               5.0        21.0           4.0   
19156              1.0               5.0        20.0           5.0   
19157              4.0               2.0        11.0           5.0   

       company_type  last_new_job  city_development_index  training_hours  
0               1.0           0.0                   0.920              36  
1               5.0           4.0                   0.776              47  
2               5.0           0.0                   0.624              83  
3               5.0           1.0                   0.789              52  
4               1.0           3.0                   0.767               8  
...             ...           ...                     ...             ...  
19153           5.0           0.0                   0.878              42  
19154           5.0           3.0                   0.920              52  
19155           5.0           3.0                   0.920              44  
19156           5.0           1.0                   0.802              97  
19157           5.0           0.0                   0.855             127  

[19158 rows x 12 columns]

In [22]:
X = new_train
y = train['target']
# Train/Test(val)분리 
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=1)

In [23]:
# from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE

# oversample = SMOTE()
# sm = SMOTE(ratio='auto', kind='regular')
sm = SMOTE(random_state=0)
X_train, y_train = sm.fit_resample(X_train, list(y_train))

In [24]:
# train/validation 분리
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.25, stratify = y_train, random_state = 1)

In [25]:
tree = DecisionTreeClassifier(max_depth=9, random_state=1)
tree.fit(X_resampled, y_resampled)

DecisionTreeClassifier(max_depth=9, random_state=1)

In [26]:
pred_train = tree.predict(X_train)
pred_test = tree.predict(X_test)

In [27]:
# 검증
from sklearn.metrics import accuracy_score
pred_train = tree.predict(X_train)
pred_val = tree.predict(X_val)

print("train 정확도:", accuracy_score(y_train, pred_train))
print("val 정확도:", accuracy_score(y_val, pred_val))

train 정확도: 0.8031773505594363
val 정확도: 0.8075282773966252


In [28]:
# 검증
accuracy_score(y_train, pred_train), accuracy_score(y_test, pred_test)

(0.8031773505594363, 0.7705636743215031)

In [29]:
print('🤔train data report')
print(classification_report(y_train, pred_train))
print('🤔test data report')
print(classification_report(y_test, pred_test))

🤔train data report
              precision    recall  f1-score   support

         0.0       0.75      0.92      0.82      8089
         1.0       0.89      0.69      0.78      8088

    accuracy                           0.80     16177
   macro avg       0.82      0.80      0.80     16177
weighted avg       0.82      0.80      0.80     16177

🤔test data report
              precision    recall  f1-score   support

         0.0       0.81      0.90      0.86      3596
         1.0       0.56      0.37      0.44      1194

    accuracy                           0.77      4790
   macro avg       0.69      0.64      0.65      4790
weighted avg       0.75      0.77      0.75      4790



In [30]:
print(classification_report(y_train, pred_train))

              precision    recall  f1-score   support

         0.0       0.75      0.92      0.82      8089
         1.0       0.89      0.69      0.78      8088

    accuracy                           0.80     16177
   macro avg       0.82      0.80      0.80     16177
weighted avg       0.82      0.80      0.80     16177



In [31]:
print('🤔train confusion matrix')
print(confusion_matrix(y_train, pred_train))
print('🤔test confusion matrix')
print(confusion_matrix(y_test, pred_test))

🤔train confusion matrix
[[7406  683]
 [2501 5587]]
🤔test confusion matrix
[[3251  345]
 [ 754  440]]


In [32]:
# 검증
accuracy_score(y_train, pred_train), accuracy_score(y_test, pred_test)

(0.8031773505594363, 0.7705636743215031)

In [33]:
from sklearn.ensemble import RandomForestClassifier
# ohe_train = ohe_train.sample(frac = 1, random_state = 0)
# X = ohe_train

# y = train['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify = y, random_state = 1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.25, stratify = y_train, random_state = 1)

rf = RandomForestClassifier(n_estimators=200, #DecisionTree 개수
                            max_features=7, #각각의 tree학습시 전달할 feature(컬럼)의 개수. 
                            max_depth=5,
                            n_jobs=-1, 
                            random_state=1)
rf.fit(X_train,y_train)

RandomForestClassifier(max_depth=5, max_features=7, n_estimators=200, n_jobs=-1,
                       random_state=1)

In [34]:
pred_train = rf.predict(X_train)
pred_test = rf.predict(X_test)

In [35]:
accuracy_score(y_train, pred_train), accuracy_score(y_test, pred_test)

(0.785192274230033, 0.7805323590814196)